# Plannings Modeling

This notebook aims at showing our modeling of the planning problem.

In [72]:
import os
import json

import numpy as np
import gurobipy as grb
from gurobipy import GRB

In [73]:
model = grb.Model()

In [74]:
# instance = "toy_instance.json"
# instance = "medium_instance.json"
instance = "large_instance.json"

file = os.path.join("instances", instance)
data = json.load(open(file, "r"))

## Parameters of the problem

Define the length of indices

In [75]:
# Number of workers
worker_length = len(data["staff"])

# Number of jobs
job_length = len(data["jobs"])

# Number of skills
skill_length = len(data["qualifications"])

# Number of days
day_length = data["horizon"]

Define jobs parameters

In [76]:
gains_job = np.array([job["gain"] for job in data["jobs"]])
penalties_job = np.array([job["daily_penalty"] for job in data["jobs"]])
due_dates_job = np.array([job["due_date"] for job in data["jobs"]])
work_days_job_skill = np.array([
    [
        job["working_days_per_qualification"][skill] if skill in job["working_days_per_qualification"] else 0
        for skill in data["qualifications"]
    ]
    for job in data["jobs"]
])

Define staff parameters

In [77]:
qualifications_worker_skill = np.array([
    [
        1 if skill in worker["qualifications"] else 0
        for skill in data["qualifications"]
    ]
    for worker in data["staff"]
])
vacations_worker_day = np.array([
    [
        1 if 1 + day in worker["vacations"] else 0
        for day in range(day_length)
    ]
    for worker in data["staff"]
])

## Decision variables

Variable to model the full solution

In [78]:
# 4-D array of binary variables : 1 if a worker is assigned to a certain project for a certain skill on a certain day, else 0
works_worker_job_skill_day = model.addVars(worker_length, job_length, skill_length, day_length, vtype=GRB.BINARY, name="work")

Variables to compute the total gain

In [79]:
is_realized_job = model.addVars(job_length, vtype=GRB.BINARY, name="is_realized") # 1 if a job is realized, else 0

Variable to compute the maximum duration (and the penalties)

In [80]:
started_after_job_day = model.addVars(job_length, day_length, vtype=GRB.BINARY, name="started_after") # 1 if a job is started after a certain day, else 0
finished_before_job_day = model.addVars(job_length, day_length, vtype=GRB.BINARY, name="finished_before") # 1 if a job is finished before a certain day, else 0
max_duration = model.addVar(vtype=GRB.INTEGER, name="max_duration") # Integer that represents the maximum duration for any job

Variables to compute the maximum assignement

In [81]:
is_assigned_worker_job = model.addVars(worker_length, job_length, vtype=GRB.BINARY, name="is_assigned") # 1 if a certain worker is assigned on a certain job, else 0
max_assigned = model.addVar(vtype=GRB.INTEGER, name="max_assigned") # Integer that represents the maximum number of assigned jobs for any worker

## Constraints

### Time Table Constraints

Define the constraints of the planning problem itself.

- Worker qualification constraint

In [82]:
model.addConstrs(
    (
        works_worker_job_skill_day[worker, job, skill, day] <= qualifications_worker_skill[worker, skill]
        for worker in range(worker_length)
        for job in range(job_length)
        for skill in range(skill_length)
        for day in range(day_length)
    ),
    name="qualification",
)

{(0, 0, 0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 0, 7): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 0, 8): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 0, 9): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 0, 10): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 0, 11): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 0, 12): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 0, 13): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 0, 14): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 0, 15): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 0, 16): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 0, 17): <gurobi.Constr *Awaiting 

- Uniqueness of the daily assignement : *done with the vacation constraint (see below)*

- Vacation constraint

In [83]:
model.addConstrs(
    (
        grb.quicksum(works_worker_job_skill_day[worker, job, skill, day] for job in range(job_length) for skill in range(skill_length)) \
            <= 1 - vacations_worker_day[worker, day]
        for worker in range(worker_length)
        for day in range(day_length)
    ),
    name="vacation",
)

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (0, 7): <gurobi.Constr *Awaiting Model Update*>,
 (0, 8): <gurobi.Constr *Awaiting Model Update*>,
 (0, 9): <gurobi.Constr *Awaiting Model Update*>,
 (0, 10): <gurobi.Constr *Awaiting Model Update*>,
 (0, 11): <gurobi.Constr *Awaiting Model Update*>,
 (0, 12): <gurobi.Constr *Awaiting Model Update*>,
 (0, 13): <gurobi.Constr *Awaiting Model Update*>,
 (0, 14): <gurobi.Constr *Awaiting Model Update*>,
 (0, 15): <gurobi.Constr *Awaiting Model Update*>,
 (0, 16): <gurobi.Constr *Awaiting Model Update*>,
 (0, 17): <gurobi.Constr *Awaiting Model Update*>,
 (0, 18): <gurobi.Constr *Awaiting Model Update*>,
 (0, 19): <gurobi.Constr *Awaiting Model 

- Job coverage constraint

In [84]:
model.addConstrs(
    (
        grb.quicksum(works_worker_job_skill_day[worker, job, skill, day] for worker in range(worker_length) for day in range(day_length)) \
            == is_realized_job[job] * work_days_job_skill[job, skill]
        for job in range(job_length)
        for skill in range(skill_length)
    ),
    name="job_coverage",
)

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (0, 7): <gurobi.Constr *Awaiting Model Update*>,
 (0, 8): <gurobi.Constr *Awaiting Model Update*>,
 (0, 9): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 9): <gurobi.Constr *Awaiting Model Update*>,


- Uniqueness of the realized projects : *done with is_realized_job either 0 or 1*

### Variable Constraints

Define the constraints that are related to the definition of additional variables.

- Is realized : *done with job coverage constraint*

- Started after / Finished before

*Example of what is expected :* 

| Day             | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 |
|-----------------|---|---|---|---|---|---|---|---|
| Working day     | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 0 |
| Started after   | 0 | 0 | 1 | 1 | 1 | 1 | 1 | 1 |
| Finished before | 0 | 0 | 0 | 0 | 0 | 1 | 1 | 1 |

In [85]:
# started_after == 0 => works == 0
model.addConstrs(
    (
        works_worker_job_skill_day[worker, job, skill, day] <= started_after_job_day[job, day]
        for worker in range(worker_length)
        for job in range(job_length)
        for skill in range(skill_length)
        for day in range(day_length)
    ),
    name="started_after",
)
# increasing sequence
model.addConstrs(
    (
        started_after_job_day[job, day] <= started_after_job_day[job, day + 1]
        for job in range(job_length)
        for day in range(day_length - 1)
    ),
    name="started_after_increasing",
)
# is_realized_job == 0 => started_after == 1
model.addConstrs(
    (
        1 - started_after_job_day[job, day] <= is_realized_job[job]
        for job in range(job_length)
        for day in range(day_length)
    ),
    name="started_after_not_realized",
)

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (0, 7): <gurobi.Constr *Awaiting Model Update*>,
 (0, 8): <gurobi.Constr *Awaiting Model Update*>,
 (0, 9): <gurobi.Constr *Awaiting Model Update*>,
 (0, 10): <gurobi.Constr *Awaiting Model Update*>,
 (0, 11): <gurobi.Constr *Awaiting Model Update*>,
 (0, 12): <gurobi.Constr *Awaiting Model Update*>,
 (0, 13): <gurobi.Constr *Awaiting Model Update*>,
 (0, 14): <gurobi.Constr *Awaiting Model Update*>,
 (0, 15): <gurobi.Constr *Awaiting Model Update*>,
 (0, 16): <gurobi.Constr *Awaiting Model Update*>,
 (0, 17): <gurobi.Constr *Awaiting Model Update*>,
 (0, 18): <gurobi.Constr *Awaiting Model Update*>,
 (0, 19): <gurobi.Constr *Awaiting Model 

In [86]:
# finished before == 1 => works == 0
model.addConstrs(
    (
        works_worker_job_skill_day[worker, job, skill, day] <= 1 - finished_before_job_day[job, day]
        for worker in range(worker_length)
        for job in range(job_length)
        for skill in range(skill_length)
        for day in range(day_length)
    ),
    name="finished_before",
)
# increasing sequence
model.addConstrs(
    (
        finished_before_job_day[job, day] <= finished_before_job_day[job, day + 1]
        for job in range(job_length)
        for day in range(day_length - 1)
    ),
    name="finished_before_increasing",
)
# is_realized_job == 0 => finished_before == 1
model.addConstrs(
    (
        1 - finished_before_job_day[job, day] <= is_realized_job[job]
        for job in range(job_length)
        for day in range(day_length)
    ),
    name="finished_before_not_realized",
)

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (0, 7): <gurobi.Constr *Awaiting Model Update*>,
 (0, 8): <gurobi.Constr *Awaiting Model Update*>,
 (0, 9): <gurobi.Constr *Awaiting Model Update*>,
 (0, 10): <gurobi.Constr *Awaiting Model Update*>,
 (0, 11): <gurobi.Constr *Awaiting Model Update*>,
 (0, 12): <gurobi.Constr *Awaiting Model Update*>,
 (0, 13): <gurobi.Constr *Awaiting Model Update*>,
 (0, 14): <gurobi.Constr *Awaiting Model Update*>,
 (0, 15): <gurobi.Constr *Awaiting Model Update*>,
 (0, 16): <gurobi.Constr *Awaiting Model Update*>,
 (0, 17): <gurobi.Constr *Awaiting Model Update*>,
 (0, 18): <gurobi.Constr *Awaiting Model Update*>,
 (0, 19): <gurobi.Constr *Awaiting Model 

- Maximum duration

In [87]:
model.addConstrs(
    (
        grb.quicksum(
            started_after_job_day[job, day] - finished_before_job_day[job, day]
            for day in range(day_length)
        ) <= max_duration
        for job in range(job_length)
    ),
    name="max_duration",
)

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>,
 21: <gurobi.Constr *Awaiting Model Update*>

- Is assigned

In [88]:
# exists_skill_day works == 1 => is_assigned == 1
model.addConstrs(
    (
        works_worker_job_skill_day[worker, job, skill, day] <= is_assigned_worker_job[worker, job]
        for worker in range(worker_length)
        for job in range(job_length)
        for skill in range(skill_length)
        for day in range(day_length)
    ),
    name="is_assigned_worker_job",
)
# forall_skill_day works == 0 => is_assigned == 0
model.addConstrs(
    (
        is_assigned_worker_job[worker, job] <= \
            grb.quicksum(
                works_worker_job_skill_day[worker, job, skill, day]
                for skill in range(skill_length)
                for day in range(day_length)
            )
        for worker in range(worker_length)
        for job in range(job_length)
    ),
    name="is_assigned_worker_job_bis",
)

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (0, 7): <gurobi.Constr *Awaiting Model Update*>,
 (0, 8): <gurobi.Constr *Awaiting Model Update*>,
 (0, 9): <gurobi.Constr *Awaiting Model Update*>,
 (0, 10): <gurobi.Constr *Awaiting Model Update*>,
 (0, 11): <gurobi.Constr *Awaiting Model Update*>,
 (0, 12): <gurobi.Constr *Awaiting Model Update*>,
 (0, 13): <gurobi.Constr *Awaiting Model Update*>,
 (0, 14): <gurobi.Constr *Awaiting Model Update*>,
 (0, 15): <gurobi.Constr *Awaiting Model Update*>,
 (0, 16): <gurobi.Constr *Awaiting Model Update*>,
 (0, 17): <gurobi.Constr *Awaiting Model Update*>,
 (0, 18): <gurobi.Constr *Awaiting Model Update*>,
 (0, 19): <gurobi.Constr *Awaiting Model 

- Maximum assigned

In [89]:
model.addConstrs(
    (
        grb.quicksum(is_assigned_worker_job[worker, job] for job in range(job_length)) \
            <= max_assigned
        for worker in range(worker_length)
    ),
    name="max_assigned",
)

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>}

## Objectives

In [90]:
# Add primary objective
model.ModelSense = GRB.MAXIMIZE
model.setObjectiveN(
  grb.quicksum(
    gains_job[job] * is_realized_job[job] \
      - penalties_job[job] * grb.quicksum(
        1 - finished_before_job_day[job, day]
        for day in range(due_dates_job[job], day_length)
      )
    for job in range(job_length)
  ),
  0,
  priority=2,
)
# Add multi-objective functions
model.setObjectiveN(
  -max_assigned,
  1,
  priority=1,
)
model.setObjectiveN(
  -max_duration,
  2,
  priority=0,
)

## Computation of a solution

In [91]:
model.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: AMD Ryzen 5 3500U with Radeon Vega Mobile Gfx, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 220197 rows, 55977 columns and 549291 nonzeros
Model fingerprint: 0x34b156d7
Variable types: 0 continuous, 55977 integer (55975 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+00, 8e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 3 objectives ... 
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve ...
---------------------------------------------------------------------------

Presolve removed 209526 rows and 50334 columns
Presolve time: 1.46s
Presolved: 10671 rows and 5643 columns
------

In [92]:
# Get optimization status
print(model.Status == GRB.OPTIMAL, model.Status == GRB.TIME_LIMIT, model.Status == GRB.INFEASIBLE)

True False False


In [93]:
# Get objective value
objective_value = model.objVal
print(objective_value)

817.0


## Visualization

In [94]:
from src.utils import disply_worker_skills, display_work_days, display_time_table, display_objectives

In [95]:
disply_worker_skills(data)

,qualifications,vacations
Olivia,"[A, C, B]","[0, 1, 2, 3, 4]"
Liam,"[E, A, D]","[5, 6, 7, 8, 9]"
Emma,"[H, B]","[20, 21, 22, 23, 24]"
Noah,"[C, H, G, D]","[25, 26, 27, 28, 29]"
Amelia,"[E, I, J, G, F]",[]
Oliver,"[J, G, F]","[30, 31, 32]"


In [96]:
display_work_days(data)

,E,I,J,A,C,H,G,D,B,F,due date,gain,penalty
Job1,0,0,0,4,0,0,0,0,4,0,26,15,3
Job2,0,0,0,4,1,0,0,4,2,0,22,30,3
Job3,0,0,0,4,1,0,0,0,0,0,16,30,3
Job4,0,0,0,6,0,0,0,0,0,0,18,30,3
Job6,0,0,0,0,4,0,0,8,3,0,14,70,3
Job7,4,0,0,0,0,7,0,0,3,0,32,50,3
Job8,0,4,0,0,0,0,0,0,3,0,18,50,3
Job9,0,0,0,0,5,0,8,5,0,0,30,80,3
Job10,0,0,0,0,7,8,0,0,0,5,20,80,3
Job11,0,5,0,0,4,0,0,0,0,0,32,20,3


In [97]:
display_time_table(data, model)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35
Olivia,None,None,None,None,None,C,B,B,A,C,C,C,A,A,C,A,C,A,A,B,A,B,A,C,B,C,B,C,C,C,C,B,C,C,C,None
Liam,D,D,D,D,E,None,None,None,None,None,A,A,A,E,A,A,E,A,D,E,D,D,E,E,E,D,E,E,D,E,D,E,D,D,D,D
Emma,H,H,H,H,B,H,H,H,H,H,H,H,H,H,B,B,H,B,H,H,None,None,None,None,None,H,H,H,H,H,H,H,H,H,H,H
Noah,C,D,G,G,D,D,G,D,C,C,H,G,H,D,C,C,C,C,G,D,D,G,D,G,G,None,None,None,None,None,D,D,G,D,D,H
Amelia,I,I,I,F,I,I,I,E,I,F,I,I,I,I,I,I,I,I,I,I,G,G,J,I,I,I,I,I,I,I,I,I,G,G,G,J
Oliver,J,J,J,J,J,J,J,J,J,F,J,F,F,F,F,F,F,J,F,F,J,J,J,J,G,J,J,G,G,G,None,None,None,G,G,J


In [98]:
display_objectives(model)

Objective : 817.0
Max assigned : 7.0
Max duration : 11.0


## Example on a Random Instance

We have implemented a tool that can generate random instances.

In [99]:
from src.create_random_instances import create_random_instance

create_random_instance(save=True, filepath="instances/random_instance.json", horizon=5, nb_jobs=5)


{'horizon': 5,
 'qualifications': ['V', 'J', 'R'],
 'staff': [{'name': 'Bqeliwwrc',
   'qualifications': ['V', 'R', 'J'],
   'vacations': []},
  {'name': 'Qhp', 'qualifications': ['V', 'J'], 'vacations': []},
  {'name': 'Ncx', 'qualifications': ['V', 'J'], 'vacations': []},
  {'name': 'Bcznzwyt', 'qualifications': ['R', 'J'], 'vacations': []},
  {'name': 'Rlvjgzghyu', 'qualifications': ['R'], 'vacations': []},
  {'name': 'Lkm', 'qualifications': ['V'], 'vacations': [5, 1]},
  {'name': 'Zlqzv', 'qualifications': ['V', 'J'], 'vacations': []},
  {'name': 'Szlzwovu', 'qualifications': ['V', 'J', 'R'], 'vacations': [5, 1]},
  {'name': 'Oowb', 'qualifications': ['J', 'V'], 'vacations': []},
  {'name': 'Mqe', 'qualifications': ['J'], 'vacations': []}],
 'jobs': [{'name': 'Job1',
   'gain': 26,
   'due_date': 5,
   'daily_penalty': 3,
   'working_days_per_qualification': {'R': 3}},
  {'name': 'Job2',
   'gain': 51,
   'due_date': 5,
   'daily_penalty': 3,
   'working_days_per_qualification': {

To help us building models with new instances, all the variables, constraints and objectives have been written in a function. 

In [100]:
from src.utils import get_instance
from src.build_model import build_model

instance_filename = "random_instance.json"
data = get_instance(instance_filename)
model = build_model(data, with_epsilon_constraint=True)
model.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: AMD Ryzen 5 3500U with Radeon Vega Mobile Gfx, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3220 rows, 857 columns and 7856 nonzeros
Model fingerprint: 0x33a0ca04
Variable types: 0 continuous, 857 integer (855 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e-03, 7e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 2574 rows and 467 columns
Presolve time: 0.04s
Presolved: 646 rows, 390 columns, 2633 nonzeros
Variable types: 0 continuous, 390 integer (388 binary)
Found heuristic solution: objective -0.0000000

Root relaxation: objective 2.177796e+02, 1638 iterations, 0.10 seconds (0.06 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0   

In [101]:
disply_worker_skills(data)

,qualifications,vacations
Bqeliwwrc,"[V, J, R]",[]
Qhp,"[V, J]",[]
Ncx,"[V, J]",[]
Bcznzwyt,"[J, R]",[]
Rlvjgzghyu,[R],[]
Lkm,[V],"[0, 4]"
Zlqzv,"[V, J]",[]
Szlzwovu,"[V, J, R]","[0, 4]"
Oowb,"[V, J]",[]
Mqe,[J],[]


In [102]:
display_work_days(data)

,V,J,R,due date,gain,penalty
Job1,0,0,3,5,26,3
Job2,4,6,4,5,51,3
Job3,7,0,0,5,28,3
Job4,1,7,4,5,60,3
Job5,10,2,3,5,71,3


In [103]:
display_time_table(data, model)

,0,1,2,3,4
Bqeliwwrc,R,J,R,V,J
Qhp,J,J,V,V,V
Ncx,V,J,V,J,J
Bcznzwyt,J,R,R,R,R
Rlvjgzghyu,R,R,R,R,R
Lkm,None,V,None,V,None
Zlqzv,V,V,V,V,J
Szlzwovu,None,R,R,R,None
Oowb,V,V,J,None,V
Mqe,J,J,J,J,J


In [104]:
display_objectives(model)

Objective : 207.975
Max assigned : 4.0
Max duration : 5.0
